In [1]:
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

In [50]:
columns = ['coordinate', 'address', 'city', 'state', 'postal_code', 
           'ad_type', 'description', 'beds', 'bath', 'sqr_lot', 'sqft', 
           'year', 'style', 'property_type', 'price', 'currency']
cities = ['San-Francisco_CA', 'New-York_NY', 'Orlando_FL']
page = 1
cursor = [page, 0]
data = []
base_url = 'https://www.realtor.com'
base_search = 'https://www.realtor.com/realestateandhomes-search/New-York_NY/pg-{page}'
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {'intl.accept_languages': 'en, en_US'})

In [3]:
driver = webdriver.Chrome('../../drivers/chromedriver', chrome_options=options)

c:\users\raphael.fontes\projetos\kaggle\.venv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [65]:
# cursor[0] += 1
# cursor[1] = 0
while True:
    driver.get(base_search.format(page=cursor[0]))
    html = driver.page_source
    soup = BeautifulSoup(html)
    properties = soup.find_all('li', class_='component_property-card')
    if properties:
        for i, property_ in enumerate(properties[cursor[1]:]):
            try:
                url = f"{base_url}{property_.a['href']}"
            except:
                continue
            else:
                driver.get(url)
                page_source = driver.page_source
                soup = BeautifulSoup(page_source)

                lat = soup.find('meta', property='place:location:latitude')['content']
                log = soup.find('meta', property='place:location:longitude')['content']
                coordinate = (lat, log)

                address = soup.find('meta', property='og:street-address')['content']
                city = soup.find('meta', property='og:locality')['content']
                state = soup.find('meta', property='og:region')['content']
                postal_code = soup.find('meta', property='og:postal-code')['content']
                type_ = soup.find('meta', property='og:type')['content']
                description = soup.find('meta', property='og:description')['content']
                price = soup.find(class_='ldp-header-price').span.text.replace('\n', '').strip()
                
                try:
                    currency = soup.find(id='priceCurrency')['value']
                except:
                    currency = None

                property_meta = soup.find('ul', class_='property-meta')
                try:
                    beds = property_meta.select('li[data-label=property-meta-beds]')[0].span.text
                except:
                    beds = None
                
                try:
                    bath = property_meta.select('li[data-label=property-meta-bath]')[0].span.text
                except:
                    bath = None
                
                try:
                    sqft = property_meta.select('li[data-label=property-meta-sqft]')[0].span.text
                except:
                    sqft = None
                
                try:
                    sqr_lot = property_meta.select('li[data-label=property-meta-lotsize]')[0].span.text
                except:
                    sqr_lot = None

                try:
                    property_year = soup.select('li[data-label=property-year]')[0].text.replace('\n', ' ').strip()
                except: 
                    property_year = None
                
                try:
                    property_style = soup.find_all('li', class_='ldp-key-fact-item')[-1].text.replace('\n', ' ').strip()
                except:
                    property_style = None
                
                try:
                    property_type = soup.select('li[data-label=property-type]')[0].text.replace('\n', ' ').strip()
                except: 
                    property_type = None
                    
                data.append([coordinate, address, city, state, postal_code, type_, description, beds, 
                             bath, sqr_lot, sqft, property_year, property_style, property_type, price, currency])
                
                cursor[1] = i + 1
                
        cursor[0] += 1
        cursor[1] = 0
    else:
        break

TypeError: 'NoneType' object is not subscriptable

In [46]:
df = pd.DataFrame(data, columns=columns)

In [47]:
df.head()

,coordinate,address,city,state,postal_code,ad_type,description,beds,bath,sqr_lot,sqft,year,style,property_type,price,currency
0,"(37.727131, -122.426165)",89 Naples St,San Francisco,CA,94112,place,Beautiful Victorian circa 1905 with high ceili...,3,3,"1,875",None,Built 1905,Style Victorian,Type Single Family Home,"$930,000",None
1,"(37.735253, -122.432332)",3045 Castro St,San Francisco,CA,94131,place,Beautifully enchanting Glen Park home w/ wonde...,2,1,"2,265","1,193",Built 1910,Style Traditional,Type Single Family Home,"$1,249,000",None
2,"(37.73024, -122.394956)",59 Reddy St,San Francisco,CA,94124,place,Great curb appeal welcomes you to this wonderf...,3,2,"2,500","1,395",Built 1916,Style Edwardian/Arts & Crafts,Type Single Family Home,"$899,000",None
3,"(37.778982, -122.391752)",177 Townsend St Unit 425,San Francisco,CA,94107,place,"2BR 2BA 1PKG 1Storage Large Private Terrace, c...",2,2,None,"1,004",Built 2007,Style Contemporary,Type Condo/Townhome/Row Home/Co-Op,"$1,249,000",None
4,"(37.744492, -122.501256)",2262 43rd Ave,San Francisco,CA,94116,place,Located in one of SF's most desirable and in d...,3,2,"2,996","1,180",Built 1943,Style Traditional,Type Single Family Home,"$1,098,000",None


In [66]:
len(data)

231

In [67]:
cursor

[7, 2]

In [49]:
df.to_csv('data/san_francisco_-_ca.csv', index=False)